In [1]:
import os
import urllib.parse

CLIENT_ID = "77ghb2exxxxxxxxxx"
REDIRECT_URI = "WPL_AP1.xxxxxxxxxxxxx"
SCOPES = ["r_liteprofile", "r_emailaddress", "w_member_social"]

def make_auth_url():
    base = "https://www.linkedin.com/oauth/v2/authorization"
    params = {
        "response_type": "code",
        "client_id": CLIENT_ID,
        "redirect_uri": REDIRECT_URI,
        "scope": " ".join(SCOPES),
        "state": "randomstatestring123"
    }
    return f"{base}?{urllib.parse.urlencode(params)}"

print("Open this URL in a browser:")
print(make_auth_url())

Open this URL in a browser:
https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id=77ghb2exxxxxxxxxx&redirect_uri=WPL_AP1.xxxxxxxxxxxxx&scope=r_liteprofile+r_emailaddress+w_member_social&state=randomstatestring123


In [ ]:
import requests

def search_people_by_title_location(access_token: str, title: str, location: str, limit: int = 10):
    """
    Search for people based on job title and location.
    Works only with LinkedIn partner/recruiter-level APIs.
    """
    url = "https://api.linkedin.com/v2/peopleSearch"

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    payload = {
        "title": title,
        "location": location,
        "count": limit
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Search error {response.status_code}: {response.text}")

    data = response.json()

    # Normalize to a list of persons (each with a person URN)
    persons = [
        {
            "name": p.get("firstName", "") + " " + p.get("lastName", ""),
            "headline": p.get("headline", None),
            "location": p.get("locationName", None),
            "profile_urn": p.get("id", None)  # usually "urn:li:person:XXXX"
        }
        for p in data.get("elements", [])
    ]

    return persons

def get_person_details(access_token: str, person_urn: str):
    """
    Retrieve detailed information about a person
    using their profile URN from the search results.
    """
    url = f"https://api.linkedin.com/v2/people/{person_urn}"

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Detail fetch error {response.status_code}: {response.text}")

    return response.json()